### Run MAF on all 75 FBS_1.4 OpSims

In [1]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [3]:
# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
Two inputs are needed here, first is the path to the directory storing the OpSim databases, the second is the path to the directory where you want to save the metadata of the metric results. By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory.

In [4]:
# Two path are needed
# 1. The path the OpSim databases
# 2. The directory where you want to save the metric metadata
dbDir = '../../FB_db/'
outDir = '../TestOutput/' 

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [5]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

# Collect Run names to a list
dbRuns = list(opSimDbs.keys())

You can also check what OpSims are available in the directory

In [6]:
show_opsims(dbDir)

['alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db',
 'baseline_v1.4_10yrs.db',
 'dcr_nham3_v1.4_10yrs.db',
 'rolling_mod3_sdf_0.20_v1.4_10yrs.db']

#### 2. Declare some metrics to run on above OpSims

In [86]:
# Airmass Metric
maxMetric = metrics.MaxMetric('airmass')
airmassSlicer = slicers.HealpixSlicer(nside=64)
airmassConstraint = 'filter = "g"'
airmassConstraint += ' and note not like "DD%"'

maxairmassSky = metricBundles.MetricBundle(maxMetric, airmassSlicer, \
                airmassConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [87]:
# Coadd Metric
coaddMetric = metrics.Coaddm5Metric()
coaddSlicer = slicers.HealpixSlicer(nside=64)
coaddConstraint = 'filter = "y"'
coaddConstraint += ' and note not like "DD%"'

CoaddY = metricBundles.MetricBundle(coaddMetric, coaddSlicer, \
                coaddConstraint)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [88]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
maxairmassSky.setSummaryMetrics(summaryMetrics)
CoaddY.setSummaryMetrics(summaryMetrics)

In [89]:
# create a bundleDict for all metrics to run on each opSim
bundleDict = {'maxairmassSky': maxairmassSky, 'CoaddY': CoaddY}

#### 3. Loop over all OpSims in dbDir and run MAF

In [90]:
for run in dbRuns:
    
    # must set run name for each opSim to store metric data into
    # separate files
    maxairmassSky.setRunName(run)
    CoaddY.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], 'TestOutput/MetricData/', resultDbs[run])
    metricGroup.runAll()

Querying database SummaryAllProps with constraint filter = "y" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'fiveSigmaDepth']
Found 390345 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'airmass']
Found 195088 visits
Running:  ['maxairmassSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "y" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'fiveSigmaDepth']
Found 384344 visits
Running:  ['CoaddY']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and note not like "DD%" for columns ['fieldDec', 'fieldRA', 'airmass']
Found 189244 visits
Running:  ['maxairmassSk